In [3]:
!pip install biopython

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 22.4 MB/s eta 0:00:00


In [4]:

#!pip3 uninstall --yes torch torchaudio torchvision torchtext torchdata
!pip3 install torch

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

In [5]:
import os
import math
import numpy as np
import random
import logging

# Bring in PyTorch
import torch
import torch.nn as nn
import torch.optim as optim
# Most of the examples have typing on the signatures for readability
from typing import Optional, Callable, List, Tuple
from Bio import SeqIO
# For data loading
from torch.utils.data import Dataset, IterableDataset, TensorDataset, DataLoader
import json
import glob
import gzip
import bz2

# For progress and timing
from tqdm import tqdm
import time
import shutil
from Bio.PDB import PDBList
from Bio.PDB.MMCIFParser import MMCIFParser
import re

#phi and psi
from Bio.PDB import PICIO, PDBIO
from Bio import PDB
from typing import TypedDict, Dict, Tuple

from Bio import Entrez
from Bio.PDB import PDBList
import pandas as pd

from transformers import BertModel, BertTokenizer
import re

from torch.utils.data import Dataset, DataLoader
from torch.utils.data import TensorDataset, DataLoader, random_split

import pickle
import shutil
from google.colab import files
from Bio.PDB import PDBParser
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer

In [6]:

tokenizer = BertTokenizer.from_pretrained("Rostlab/prot_bert", do_lower_case=False ) # change model and tokenizer to t5
model_embedd = BertModel.from_pretrained("Rostlab/prot_bert")



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/86.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/81.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/361 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.68G [00:00<?, ?B/s]

In [7]:
angles = torch.load('/content/angles.pt')
phi_angles = angles[:,0,:]
psi_angles = angles[:,1,:]

print(angles.size())
print(phi_angles.size())
print(angles[0,:,:])

torch.Size([40, 2, 130])
torch.Size([40, 130])
tensor([[-179.7694, -179.8116, -179.9852, -179.7527, -179.8434, -179.9999,
         -179.9141,  179.8694, -179.8891,  179.8557, -179.9297, -179.9924,
          179.9637,  179.9209,  179.9829,  179.9279, -179.9967,  179.9088,
         -179.8918,  179.9011,  179.9906,  179.8926, -179.9534, -179.9581,
          179.9089, -179.8966, -179.9512, -179.9711, -179.9718,  179.9720,
         -179.9818,  179.9500, -179.9891,  179.8605,  179.8819,  179.8916,
          179.8811,  179.9427,  179.9055, -179.9232,  179.9547,  179.8050,
          179.9984,  179.8610, -179.9773,  179.9883,  179.9742, -179.9785,
         -179.9120, -179.9093, -179.9617,  179.8879,  179.7969, -179.9996,
          179.9560, -179.9032, -179.9032,  179.9633, -179.9339,  179.9616,
         -179.8803,  179.9585, -179.9204, -179.8699, -179.9196,  179.9830,
          179.8819,  179.9019,  179.9564,  179.8550, -179.9350,  179.9513,
          179.8102,  179.7916, -179.9457,  179.9998, 

In [8]:
embeddings = torch.load('/content/embeddings.pt')
print(embeddings.size())

torch.Size([40, 130, 1024])


In [9]:

  # Ensure this matches the file path used when saving
with open('/content/sequences.pkl', 'rb') as f:
        sequences = pickle.load(f)
print(sequences)
print(len(sequences)) #here len is not the length of sequence, but number of sequences in total

['G S S G S S G N M T I S D I Q H A P D Y A L L S N D E Q Q L C I Q L K I L P K P Y L V L K E V M F R E L L K T G G N L S K S A C R E L L N I D P I K A N R I Y D F F Q S Q N W M', 'G A M G M P N R K Y A D G E V V M G R W P G S V L Y Y E V Q V T S Y D D A S H L Y T V K Y K D G T E L A L K E S D I R L Q S S F K Q', 'G A M D P E F N K A I F T V D A K T T E I L V A N D K A C G L L G Y S S Q D L I G Q K L T Q F F L R S D S D V V E A L S E E H M E A D G H A A V V F G T V V D I I S R S G E K I P V S V W M K R M R Q E R R L C C V V V L E P V E R', 'G S S G S S G A G Q V V H T E T T E V V L T A D P V T G F G I Q L Q G S V F A T E T L S S P P L I S Y I E A D S P A E R C G V L Q I G D R V M A I N G I P T E D S T F E E A N Q L L R D S S I T S K V T L E I E F D V A E S V I P S S G S G P S S G', 'A M F Q I G K M R Y V S V R D F K G K V L I D I R E Y W M D P E G E M K P G R K G I S L N P E Q W S Q L K E Q I S D I D D A V R K L', 'G S S G S S G L E T S S Y L N V L V N S Q W K S R W C S V R D N H L H F

In [12]:

class SelfAttention(nn.Module):
    def __init__(self, embed_dim: int):
        super(SelfAttention, self).__init__()
        self.embed_dim = embed_dim
        self.w_q = nn.Parameter(torch.randn(embed_dim, embed_dim))
        self.w_k = nn.Parameter(torch.randn(embed_dim, embed_dim))
        self.w_v = nn.Parameter(torch.randn(embed_dim, embed_dim))

    def forward(self, embeddings_prot_bert: torch.Tensor) -> torch.Tensor:
        Q = torch.matmul(embeddings_prot_bert, self.w_q)
        K = torch.matmul(embeddings_prot_bert, self.w_k)
        V = torch.matmul(embeddings_prot_bert, self.w_v)

        scores = torch.matmul(Q, K.transpose(-1, -2)) / np.sqrt(K.size(-1))
        attn = torch.softmax(scores, dim=-1)
        attention_output = torch.matmul(attn, V)

        return attention_output


class TransformerModel(nn.Module):
    def __init__(self, embed_dim: int, feed_forward_dim1: int, feed_forward_dim2: int, output_dim: int = 2, dropout_rate: float = 0.1):
        super(TransformerModel, self).__init__()
        self.self_attention = SelfAttention(embed_dim)
        self.layer_norm1 = nn.LayerNorm(embed_dim)
        self.layer_norm2 = nn.LayerNorm(output_dim)
        self.dropout = nn.Dropout(dropout_rate)
        self.feed_forward = nn.Sequential(
            nn.Linear(embed_dim, feed_forward_dim1),
            nn.GELU(),
            self.dropout,
            nn.Linear(feed_forward_dim1, feed_forward_dim2),
            nn.GELU(),
            self.dropout,
            nn.Linear(feed_forward_dim2, output_dim)
        )

    def forward(self, embeddings: torch.Tensor) -> torch.Tensor:
        attention_output = self.self_attention(embeddings)
        normalized_attention_output = self.layer_norm1(attention_output)
        ff_output = self.feed_forward(normalized_attention_output)
        output = self.layer_norm2(ff_output)
        return ff_output
class AngularLoss(nn.Module):
    def __init__(self):
        super(AngularLoss, self).__init__()

    def forward(self,idx, predicted_angles, angles_tensor):

        predicted_angles_phi, predicted_angles_psi = predicted_angles[:, 0], predicted_angles[:, 1]
        angles_tensor_phi, angles_tensor_psi = angles_tensor[:,0, idx], angles_tensor[:, 1,idx]

        predicted_angles_phi = (predicted_angles_phi + torch.pi) % (2 * torch.pi) - torch.pi
        angles_tensor_phi = (angles_tensor_phi + torch.pi) % (2 * torch.pi) - torch.pi
        predicted_angles_psi = (predicted_angles_psi + torch.pi) % (2 * torch.pi) - torch.pi
        angles_tensor_psi = (angles_tensor_psi + torch.pi) % (2 * torch.pi) - torch.pi

        difference_phi = torch.abs(predicted_angles_phi - angles_tensor_phi)
        loss_phi = torch.mean(torch.min(difference_phi, 2 * torch.pi - difference_phi))

        difference_psi = torch.abs(predicted_angles_psi - angles_tensor_psi)
        loss_psi = torch.mean(torch.min(difference_psi, 2 * torch.pi - difference_psi))

        loss = loss_phi + loss_psi
        #print(loss)
        return loss

class TransformerTrainer:
    def __init__(self, model: nn.Module, criterion: nn.Module, num_epochs: int, sequence: torch.Tensor, angles: torch.Tensor):
        self.model = model
        self.criterion = criterion
        self.num_epochs = num_epochs
        self.sequence = sequence
        self.angles_tensor = angles
        self.optimizer = optim.AdamW(model.parameters(), lr=0.001)
        #self.scheduler = optim.lr_scheduler.StepLR(self.optimizer, step_size=10, gamma=0.1)

    def train(self):
        loss_list = []
        indices = np.arange(0, len(self.sequence[:,0,0])) # shuffling the indices around, maybe we can do it after the epoch for more shuffling
        random.shuffle(indices)
        for epoch in range(self.num_epochs):
          for idx in indices[:int(0.8*len(self.sequence[:,0,0]))]: #im taking 80 percent of the dataset
            self.optimizer.zero_grad()
            predictions = self.model.forward(self.sequence[idx,:,:])[:, :len(self.angles_tensor[:,:,idx])]
            #loss = CustomLoss()
            #loss.forward(torch.tensor(predictions, requires_grad=True), self.angles_tensor)
            loss = self.criterion(idx,predictions.squeeze(), self.angles_tensor)
            loss.backward(retain_graph=True)
            self.optimizer.step()
            #self.scheduler.step()
            loss_list.append(loss.item())

          print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {loss.item()}")
        return indices
        #return loss


In [13]:
#source_dir = '/content/map'

#tokenizer = BertTokenizer.from_pretrained("Rostlab/prot_bert", do_lower_case=False )
# Initialize and train transformer model
feed_forward_dim1 = 512
feed_forward_dim2 = 256
feed_forward_dim3 = 128
num_epochs = 100
dropout_rate = 0.1
model = TransformerModel(embed_dim=1024, feed_forward_dim1=feed_forward_dim1, feed_forward_dim2= feed_forward_dim2, dropout_rate = dropout_rate)
criterion = AngularLoss()

N= 130
D = 1024
indices = np.arange(0, len(sequences))
random.shuffle(indices) # shuffling data

#for i in indices: # we can replace this with something better once we get all the files
#  angles_tensor = (angles[i,:,:].T)*(np.pi/180)
#  #mask = create_mask(torch.tensor([130]), 130)
#  trainer = TransformerTrainer(model, criterion, num_epochs, embeddings[i,:,:] , angles_tensor)#, mask)
#  print(i)
#  trainer.train()

angles_tensor = (angles.T)*(np.pi/180)
trainer = TransformerTrainer(model, criterion, num_epochs, embeddings , angles_tensor)#, mask)
trainer.train()

<ipython-input-13-43bd0a1002cf>:25: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matrices or `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3675.)
  angles_tensor = (angles.T)*(np.pi/180)


Epoch 1/100, Loss: 1.9283802509307861
Epoch 2/100, Loss: 1.4534051418304443
Epoch 3/100, Loss: 2.269693374633789
Epoch 4/100, Loss: 1.6314780712127686


KeyboardInterrupt: 